# Seldon Core Setup

## Prerequisites

You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [A running Kubernetes cluster with kubectl authenticated](https://kubernetes.io/docs/setup/)
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core>=0.2.6.1```)
 - [Helm client](https://helm.sh/)

## Setup Cluster

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Install Seldon Core

Follow the [Seldon Core Install documentation](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html).

In [ ]:
!kubectl create namespace seldon-system

### If using Ambassador:

In [ ]:
!helm install seldon-core seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --set ambassador.enabled=true --set usageMetrics.enabled=true --namespace seldon-system

### If using Istio:

In [ ]:
!helm install seldon-core seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --set istio.enabled=true --set usageMetrics.enabled=true --namespace seldon-system

Check all services are running before proceeding.

### Wait for rollout to finish

In [ ]:
!kubectl rollout status deploy/seldon-controller-manager -n seldon-system

## Install Ingress

### Ambassador

[Ambassador install](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html#install-ambassador)

Ambassador Edge Stack:

In [ ]:
!helm repo add datawire https://www.getambassador.io

In [ ]:
!helm repo update

In [ ]:
!helm install ambassador datawire/ambassador \
    --set image.repository=docker.io/datawire/ambassador \
    --set crds.keep=false \
    --namespace seldon-system

Ambassador API Gateway (Emissary Ingress):

In [ ]:
!helm repo add datawire https://www.getambassador.io

In [ ]:
!helm repo update

In [ ]:
!helm install ambassador datawire/ambassador \
    --set image.repository=docker.io/datawire/ambassador \
    --set crds.keep=false \
    --set enableAES=false \
    --namespace seldon-system

Check all services are running before proceeding.

In [ ]:
!kubectl rollout status deployment.apps/ambassador

For Ambassador Edge Stack:

**Note:** Absent configuration, Ambassador Edge Stack will attempt to use TLS with a self-signed cert.  Either [set up TLS](https://www.getambassador.io/docs/edge-stack/latest/howtos/tls-termination/) or ignore certificate validation (i.e. `curl -k`).

In [ ]:
!kubectl port-forward $(kubectl get pods -n seldon-system -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon-system 8003:8443

For Ambassador API Gateway:

In [ ]:
!kubectl port-forward $(kubectl get pods -n seldon-system -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon-system 8003:8080

*or*

### Istio

[Istio install](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html#ingress-support)

**Note:** Remember to add ```--set istio.enabled=true``` flag when installing Seldon Core with Istio Ingress.